<a href="https://colab.research.google.com/github/poojapandit963/study_planner/blob/main/AI_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#final code
import pandas as pd
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression

# Function to read the CSV file and build the complexity dictionary
def build_subjects_topics_complexity_dict(csv_file):
    data = pd.read_csv(csv_file)
    data.fillna(data.mean(), inplace=True)
    subjects_topics_complexity = {(str(s).lower(), str(t).lower()): c for s, t, c in zip(data['Subject'], data['Topics'], data['Complexity'])}
    return subjects_topics_complexity, data

# Train the linear regression model
def train_linear_regression_model(data):
    model = LinearRegression()
    model.fit(data[['Complexity']], data['Time'])
    return model

# Function to calculate study time based on the user input topics
def calculate_study_time(subject, topics, subjects_topics_complexity, model):
    subject = subject.lower()
    topics = [t.lower().strip() for t in topics.split(",")]

    print("Subjects-Topics Complexity Dictionary:")
    print(subjects_topics_complexity)

    relevant_topics = []
    for topic in topics:
        print(f"Looking for topic {topic} in {subject}")
        if (subject, topic) in subjects_topics_complexity:
            relevant_topics.append(topic)
        elif (subject, '') in subjects_topics_complexity:
            return subjects_topics_complexity[(subject, '')]

    if not relevant_topics:
        print(f"No relevant topics found for {subject}.")
        return 0

    print("Relevant Topics:")
    print(relevant_topics)

    complexity = sum([subjects_topics_complexity[(subject, topic)] for topic in relevant_topics])

    print(f"Complexity for {subject}: {complexity}")

    if complexity > 0:
        return model.predict([[complexity]])[0] * complexity
    else:
        return 0
def display_schedule(schedule):
    print("{:<20} {:<40} {:<20} {:<20}".format('Subject', 'Topics', 'Study Time', 'Time Slot'))
    for entry in schedule:
        print("{:<20} {:<40} {:<20} {:<20}".format(entry[0], entry[1], entry[2], entry[3]))



# Function to schedule the study timetable with breaks and user constraints
def schedule_study_timetable(subjects, start_time, end_time, break_time, lunch_start_time, lunch_end_time, unavailable_time_slots, subjects_topics_complexity, model):
    current_time = start_time
    lunch_time_slot = (lunch_start_time, lunch_end_time)
    print(f"Lunch Time: {lunch_time_slot[0].strftime('%I:%M%p')} - {lunch_time_slot[1].strftime('%I:%M%p')}")
    print(f"Break Time: {break_time} hours")
    print("Unavailable Time Slots:")
    for slot in unavailable_time_slots:
        print(slot)
    print("Scheduled Study Time:")
    scheduled = []
    for subject in subjects:
        topics = input(f"Enter the topics for {subject} (separated by comma): ")
        study_time = calculate_study_time(subject, topics, subjects_topics_complexity, model)
        if study_time:
            print(f"Subject: {subject} - Topics: {topics}")
            print(f"Study Time: {study_time:.2f} hours")
            end_study_time = current_time + timedelta(hours=study_time)

            if end_study_time > end_time:
                print("End time reached. Cannot schedule further.")
                break

            while current_time.strftime('%I%p') in unavailable_time_slots or end_study_time.strftime('%I%p') in unavailable_time_slots or (current_time < lunch_time_slot[1] and end_study_time > lunch_time_slot[0]) or (current_time < end_time and end_study_time > end_time):
                current_time += timedelta(hours=1)
                end_study_time = current_time + timedelta(hours=study_time)

                if end_study_time > end_time:
                    print("End time reached. Cannot schedule further.")
                    break

            print(f"Scheduled from {current_time.strftime('%I:%M%p')} to {end_study_time.strftime('%I:%M%p')}")

            scheduled.append([subject, topics, f"{study_time:.2f} hours", f"{current_time.strftime('%I:%M%p')} - {end_study_time.strftime('%I:%M%p')}"])

            current_time = end_study_time + timedelta(hours=break_time)
            if current_time >= end_time:
                print("End time reached. Scheduled the whole available time.")
                break

            print()

        else:
            print(f"Cannot calculate study time for {subject}.")

    if current_time < end_time:
        scheduled.append(["Available Time", "-", f"{(end_time - current_time).seconds // 3600} hours", f"{current_time.strftime('%I:%M%p')} - {end_time.strftime('%I:%M%p')}"])
        print("Scheduled the whole available time.")

    display_schedule(scheduled)


# Example usage for the study planner
subjects_topics_complexity, data = build_subjects_topics_complexity_dict('study_data.csv')
model = train_linear_regression_model(data)

user_subjects = input("Enter the subjects separated by a comma: ").split(",")
user_start_time = datetime.strptime(input("Enter the start time of study (e.g., 8AM): "), '%I%p')
user_end_time = datetime.strptime(input("Enter the end time of study (e.g., 6PM): "), '%I%p')
user_break_time = float(input("Enter break time in hours (e.g., 1.5): "))
user_lunch_start_time = datetime.strptime(input("Enter the lunch start time (e.g., 12PM): "), '%I%p')
user_lunch_end_time = datetime.strptime(input("Enter the lunch end time (e.g., 1PM): "), '%I%p')
unavailable_time_slots = [input("Enter unavailable time slot (e.g., 2PM - 3PM): ")]

# Call the functions with user inputs
schedule_study_timetable(user_subjects, user_start_time, user_end_time, user_break_time, user_lunch_start_time, user_lunch_end_time, unavailable_time_slots, subjects_topics_complexity, model)


<ipython-input-2-2ce28dd6ffbe>:9: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.fillna(data.mean(), inplace=True)


Enter the subjects separated by a comma: ins
Enter the start time of study (e.g., 8AM): 8AM
Enter the end time of study (e.g., 6PM): 9PM
Enter break time in hours (e.g., 1.5): 1.0
Enter the lunch start time (e.g., 12PM): 1PM
Enter the lunch end time (e.g., 1PM): 2PM
Enter unavailable time slot (e.g., 2PM - 3PM): 4PM - 5PM
Lunch Time: 01:00PM - 02:00PM
Break Time: 1.0 hours
Unavailable Time Slots:
4PM - 5PM
Scheduled Study Time:
Enter the topics for ins (separated by comma): sdes,rsa
Subjects-Topics Complexity Dictionary:
{('ai', 'a*'): 1, ('ai', 'hil_climbing'): 1, ('ai', 'bfs'): 2, ('ai', 'dfs'): 1, ('ai', 'greedy_best_first'): 1, ('ra', 'asimov law'): 2, ('ra', 'dof'): 1, ('ra', 'components of robots'): 2, ('ra', 'application'): 1, ('ra', 'joints'): 1, ('eo', 'modi'): 1, ('eo', 'vam'): 2, ('eo', 'trans'): 1, ('eo', 'two-phase'): 2, ('eo', 'simplex'): 2, ('wt', 'padding'): 3, ('wt', 'javascript'): 2, ('wt', 'php'): 3, ('wt', 'fonts'): 2, ('wt', 'html'): 1, ('wt', 'css'): 2, ('wt', 'ap

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
